In [1]:
%%capture
!pip install langchain
!pip install openai
!pip install chroma
!pip install chromadb
!pip install langchain_community
!pip install tiktoken
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install PyPDF2 Pillow
!pip install pdf2image
!sudo apt install poppler-utils
!sudo apt install tesseract-ocr-spa
!tesseract -l spa

In [2]:
from langchain_community.document_loaders import Docx2txtLoader #To load word files
from langchain.document_loaders import PyPDFLoader #To load pdf files
from langchain.vectorstores import Chroma #Vectorial DataBase
from langchain.embeddings.openai import OpenAIEmbeddings #Word2Vec Model to tokenizer the text
from langchain.text_splitter import RecursiveCharacterTextSplitter #To splitt the text
from langchain.chat_models import ChatOpenAI #To load LLM from OPENAI
from langchain.chains import RetrievalQA #To build chains to QA tasks
from langchain import PromptTemplate #Class that allow the before to get in in production
from langchain.chains.summarize import load_summarize_chain #To buil chains to summarize tasks
import os
from langchain.vectorstores import Chroma #The vectorial database
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter #To splitt the text
from langchain.embeddings.openai import OpenAIEmbeddings #Word2Vec Model to tokenizer the text
from langchain.vectorstores import Chroma #The vectorial database
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import PyPDF2
from PIL import Image
import io
from pdf2image import convert_from_path
import cv2
import pytesseract
import os

## PDF to images


In [23]:
def pdf_to_images(path, output_folder="Imagenes"):
    """
    Converts a PDF file into images and saves them in the specified output folder.

    Args:
        path (str): The file path to the PDF.
        output_folder (str): The folder where the images will be saved. Default is "Imagenes".

    Returns:
        None
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pages = convert_from_path(path)

    for i, page in enumerate(pages):
        image_name = os.path.join(output_folder, f"Page_{i+1}.jpg")
        page.save(image_name, "JPEG")
        print(f"Saved: {image_name}")

In [24]:
pdf_path = "/content/Bruno_child_offers.pdf"
pdf_to_images(pdf_path)

Saved: Imagenes/Page_1.jpg
Saved: Imagenes/Page_2.jpg
Saved: Imagenes/Page_3.jpg
Saved: Imagenes/Page_4.jpg
Saved: Imagenes/Page_5.jpg
Saved: Imagenes/Page_6.jpg
Saved: Imagenes/Page_7.jpg
Saved: Imagenes/Page_8.jpg
Saved: Imagenes/Page_9.jpg
Saved: Imagenes/Page_10.jpg
Saved: Imagenes/Page_11.jpg
Saved: Imagenes/Page_12.jpg
Saved: Imagenes/Page_13.jpg
Saved: Imagenes/Page_14.jpg
Saved: Imagenes/Page_15.jpg
Saved: Imagenes/Page_16.jpg
Saved: Imagenes/Page_17.jpg
Saved: Imagenes/Page_18.jpg
Saved: Imagenes/Page_19.jpg
Saved: Imagenes/Page_20.jpg
Saved: Imagenes/Page_21.jpg
Saved: Imagenes/Page_22.jpg


## Images to text

In [25]:
def escribir_a_txt(contenido, nombre_archivo):
    """
    Writes the content of a string to a text file.

    Args:
        content (str): The string content to be written to the file.
        filename (str): The name of the text file.

    Returns:
        None
    """
    try:
      # Open the file in write mode ("w" for overwrite)
      with open(nombre_archivo, "w") as archivo:
        # Write the contents of the string to the file
        archivo.write(contenido)
      print(f"The content has been successfully written to the file {nombre_archivo}.")
    except FileNotFoundError:
      print(f"Error: Could not find file {nombre_archivo}.")
    except Exception as e:
      print(f"Unexpected error: {e}")

def txt_to_langchain():
    """
    Loads and concatenates text files from a specified directory into a list.

    Returns:
        list: Concatenated content of all text files.

    Notes:
        Prints status messages during the process and handles errors by printing error messages.
    """


    concatenated_files=[]
    root_path="Txt_image/"
    txt_files=list(os.listdir(root_path))
    number_pages=len(txt_files)
    for i in range(1,number_pages+1):
        try:
            print("STARTING TO CODE PAGE : ",i)
            loader = TextLoader(root_path+f"Page_{i}.txt", encoding="utf-8")
            data=loader.load()
            concatenated_files.extend(data)
            print("CODED PAGE : ",i)
        except:
            print(f"error in Page_{i}.txt")
    print("CODED DATA : ",concatenated_files)
    return concatenated_files

In [26]:
# Image upload
lista_imagenes =  os.listdir('Imagenes')

for imgPath in lista_imagenes:
  img = cv2.imread("/content/Imagenes/"+imgPath)
  # COnvertimage to gray scale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # Apply threshold to convert to binary image
  threshold_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  # Pass the image through pytesseract
  text = pytesseract.image_to_string(threshold_img,lang='spa')
  # Print the extracted text
  escribir_a_txt(text,'Txt_image/'+imgPath.split('.')[0]+'.txt')

########## READ TXT AND PASS TO LANGCHAIN ​​ITEMS

codificate_pages = txt_to_langchain()

The content has been successfully written to the file Txt_image/Page_4.txt.
The content has been successfully written to the file Txt_image/Page_8.txt.
The content has been successfully written to the file Txt_image/Page_6.txt.
The content has been successfully written to the file Txt_image/Page_14.txt.
The content has been successfully written to the file Txt_image/Page_1.txt.
The content has been successfully written to the file Txt_image/Page_16.txt.
The content has been successfully written to the file Txt_image/Page_17.txt.
The content has been successfully written to the file Txt_image/Page_5.txt.
The content has been successfully written to the file Txt_image/Page_11.txt.
The content has been successfully written to the file Txt_image/Page_22.txt.
The content has been successfully written to the file Txt_image/Page_18.txt.
The content has been successfully written to the file Txt_image/Page_9.txt.
The content has been successfully written to the file Txt_image/Page_7.txt.
The co

## TRANSFER TO A VECTORIZED DATABASE

In [27]:
def get_vdb(concatenated_files):
    """
    Converts concatenated text files into a vectorized database.

    Args:
        concatenated_files (list): List of concatenated text files.

    Returns:
        Chroma: A vector store created from the documents.

    Notes:
        - Splits text into fragments of 1500 tokens with 200 tokens overlap.
        - Uses OpenAI's text-embedding-ada-002 model for embeddings.
        - Persists the vector store in a specified directory.
    """
    #SPLITTING THE TEXTS:
    print("STARTING TO TRANSFER TO A VECTORIZED DATABASE")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500, #Fragments of text of 1500 tokens
        chunk_overlap=200, #For evey fragment that take the 200 last tokens of the last fragment
        length_function=len
        )
    documents = text_splitter.split_documents(concatenated_files) #List with the metadata and the content splitt by fragments of 1500 tokens

    print("GETTING VECTORIZED DOCUMENT LIST: ",documents)
    #CARRY THE CONTENT OF THE DOCUMENTS TO A VECTORIAL DATABASE:
    #------------------------------------------------------------
    #------------------------------------------------------------
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") #word2vec model of openAI
    print("WE GET THE EMBEDDING MODEL")
    NOMBRE_INDICE_CHROMA = f"vdb_{'Base_ejemplo3'}" #Name of my vectorial database (Put the name that you want)

    print("VECTORIZING")
    #Creating our vectorial database or vector store
    vectorstore_chroma = Chroma.from_documents(
        documents=documents, #Create the database with the list of the created documents (Every instance will be the embedding of every document)
        embedding=embeddings, #, #Word2vec model to create our embeddings, always use the same.
        persist_directory=NOMBRE_INDICE_CHROMA #Load my database in the indicated folder (If I close the section, I will keep storaged my vectorial databas in the folder called "NOMBRE_INDICE_CHROMA" )
    )
    print("VECTORIZED")
    #loaded_
    #loaded_database = Chroma(persist_directory=f"AzureStorage/CerebroAuditoria/temporal_databases/vdb_{cont}",embedding_function=embeddings)

    return vectorstore_chroma

In [28]:
################################################################
######## WE GO TO THE VECTORIZED DATABASE ################
################################################################
OPENAI_API_KEY = "YOUR KEY"
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

chroma_db = get_vdb(codificate_pages)

STARTING TO TRANSFER TO A VECTORIZED DATABASE
GETTING VECTORIZED DOCUMENT LIST:  [Document(metadata={'source': 'Txt_image/Page_1.txt'}, page_content='MA\n\na\n\nKitx 2 Frozen de Disney\n\nGel para el Cabello 65 g,\n\nprecio g $187,24 + Colonia 70 ml.\nPrecio regular\n\n(168391) 2 pts\n\nKITx2\n$21.900\n\nLlévalos también\nen venta individual:\n\nCON BRILLO\nA. Gel para el Cabello\n\nFrozen de Disney 65 g.\nPrecio regular\n\n(133996) 1 pt\n\n$13.900\n\nPrecio g $213,85\n\nB. Colonia Frozen\nde Disney 70 ml. |\nPrecio regular\n\n  \n\n(133993) 1 pt\n$12.900\n\n78 *Dermatológicamente probado. Basado en estudios técnicos realizados. ODisney'), Document(metadata={'source': 'Txt_image/Page_3.txt'}, page_content='CUIDADOS ESPECIALES\n\npara sorprender y divertirte con tus niñas.\n\n \n\nPNOTaaN\n\n \n\nCepillo Corona Diamantes\n\nFrozen de Disney\n\nPlástico y nailon. Diseño con forma\n\nde corona con simulación de Shampoo 2 en 1 Frozen Barrita Humectante\ndiamantes. Largo: 17.6 cm, ancho: de

## GENERAMOS CHAT PARA OBTENER LAS RESPUESTAS A LAS PREGUNTAS SOLICITADAS

In [29]:
answers=[]

In [30]:
#DEFINING THE CHAT CHAIN:
    #----------------------------------------------------------------
    #----------------------------------------------------------------
template = """Eres un chatbot que tiene una conversación con un humano y tu función es responder las preguntas hechas por el humano sobre el contenido del archivo PDF proporcionado.

Dado el siguiente contexto y teniendo en cuenta el historial de la conversación, responde a las preguntas hechas por el humano y entrega la información textual que está en el PDF sin cambiar lo que aparece. Si en el contexto proporcionado no está la respuesta a la pregunta hecha por el usuario, no inventes. Además, no debes responder preguntas que no estén relacionadas con el contenido del PDF, como matemáticas, religión, etc.

{context}

{chat_history}
Human: {human_input} ~~~~~
Chatbot (Responde de una manera formal):"""



prompt = PromptTemplate(
input_variables=["chat_history", "human_input", "context"], template=template
)


questions=["¿Que colonias tienes de disney",
           "¿Cual es la religión más practican en colombia?",
           "¿Hay alguna colonia o producto que sea dermatológicamente probado?", 
           "Que venden en la última página del pdf"]

for i,query in enumerate(questions):
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
  chain = load_qa_chain(
    ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0), chain_type="stuff", memory=memory, prompt=prompt
    )

  docs=chroma_db.similarity_search(query,k=6)
  response=chain({"input_documents": docs, "human_input": query, "chat_history":memory}, return_only_outputs=False)['output_text'] #AI answer
  answers.append(response)

answers

['Tenemos disponibles las colonias Disney Minnie Mouse y Disney Princesa en presentaciones de 150 ml cada una. La Colonia Minnie Mouse tiene un precio regular de $18.900 y la Colonia Disney Princesa tiene un precio regular de $25.900. Ambas colonias son dermatológicamente probadas y están libres de parabenos.',
 'Lo siento, pero no tengo información sobre la religión más practicada en Colombia en el contenido del PDF proporcionado. ¿Tienes alguna otra pregunta relacionada con el archivo?',
 'Sí, la Colonia Minnie Mouse de Disney y la Colonia Barbie Spray Desenredante Barbie son productos que han sido dermatológicamente probados, según se indica en el archivo PDF proporcionado.',
 'En la última página del PDF se encuentra la oferta de un gel para el cabello Spider-Man de Marvel de 100 g, con un precio regular de $24.990 y un descuento especial a $13.900. Este producto ha sido dermatológicamente probado y está basado en estudios técnicos realizados.']